In [ ]:
using Turing
using StatsPlots
using Optim

In [ ]:
PRINT_PROGRESS = true
include("read_data.jl")
include("utils.jl")
include("lecture/plot_utils.jl");

### Dataset: World-Wide Monthly Air Passengers

In [ ]:
base_plot(air_passengers_2013_2018)

In [ ]:
upwards_trend_data = air_passengers_2013_2018[2009 .<= air_passengers_2013_2018[!,"Year"],:]
base_plot(upwards_trend_data)

x = upwards_trend_data[!,"Date"]
y = upwards_trend_data[!,"Total"];

In [ ]:
@model function trend_model(x::Vector{Float64}, y::Vector{Float64})
    # priors are guided by data but chosen to be very broad
    # (we do not want to favour any specific values)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    # error models how noisy the data is, it has to be >0 => InverseGamma
    error ~ InverseGamma(2,3)
    
    for i in eachindex(y)
        # shift the x-axis by start year 0...16 instead of 2003...2019
        y[i] ~ Normal(slope * (x[i] - x[1]) + intercept, error+1e-5)
    end
end

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model(x, y), NUTS(), 3000, progress=PRINT_PROGRESS)
res

In [ ]:
plot(res)

In [ ]:
_, map_vector = get_map(res)
plot_trend_model_1_map(upwards_trend_data, map_vector)

In [ ]:
plot_trend_model_1_posterior(upwards_trend_data, res[1:30:end])